In [1]:
import json
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import pickle
from bs4 import BeautifulSoup
import re

/home/tommytyc/anaconda3/envs/test_Tempest/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tommytyc/anaconda3/envs/test_Tempest/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tommytyc/anaconda3/envs/test_Tempest/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tommytyc/anacond

In [2]:
import torch
import torch.nn as nn
from torch.utils import data
from torch import optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import  pack_padded_sequence, pad_packed_sequence

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import random

In [3]:
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from rouge import Rouge 
rouge = Rouge()

In [4]:
# with open("product_desc_dict.pickle", 'rb') as f:
#     prod_desc = pickle.load(f)
# with open("product_intro_dict.pickle", 'rb') as f:
#     prod_intro = pickle.load(f)
# with open("product_name_dict.pickle", 'rb') as f:
#     prod_name = pickle.load(f)

In [5]:
# with open('com_product_intro_word_articles_train.json', 'r') as f:
#     intro = json.load(f)
# with open('com_product_name_word_flatten_intro_train.json', 'r') as f:
#     name = json.load(f)

In [6]:
# def filter_tags(htmlstr):
#     re_cdata=re.compile('//<!\[CDATA\[[^>]*//\]\]>',re.I)
#     re_script=re.compile('<\s*script[^>]*>[^<]*<\s*/\s*script\s*>',re.I)
#     re_style=re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>',re.I)
#     re_br=re.compile('<br\s*?/?>')
#     re_h=re.compile('</?\w+[^>]*>')
#     re_comment=re.compile('<!--[^>]*-->')
#     s=re_cdata.sub('',htmlstr)
#     s=re_script.sub('',s)
#     s=re_style.sub('',s)
#     s=re_br.sub('',s)
#     s=re_h.sub('',s) 
#     s=re_comment.sub('',s)
#     blank_line=re.compile('\n+')
#     s=blank_line.sub('\n',s)
#     s=replaceCharEntity(s)
#     return s

# def replaceCharEntity(htmlstr):
#     CHAR_ENTITIES={'nbsp':' ','160':' ',
#                 'lt':'<','60':'<',
#                 'gt':'>','62':'>',
#                 'amp':'&','38':'&',
#                 'quot':'"','34':'"',}
#     re_charEntity=re.compile(r'&#?(?P<name>\w+);')
#     sz=re_charEntity.search(htmlstr)
#     while sz:
#         entity=sz.group()
#         key=sz.group('name')
#         try:
#             htmlstr=re_charEntity.sub(CHAR_ENTITIES[key],htmlstr,1)
#             sz=re_charEntity.search(htmlstr)
#         except KeyError:
#             htmlstr=re_charEntity.sub('',htmlstr,1)
#             sz=re_charEntity.search(htmlstr)
#     return htmlstr

In [7]:
# def data_cleaner(data) -> dict:
#     new_data = {}
#     for key in data:
#         new_value = filter_tags(data[key])
#         new_data[key] = new_value
#     return new_data

In [8]:
# prod_desc = data_cleaner(prod_desc)
# prod_intro = data_cleaner(prod_intro)
# prod_name = data_cleaner(prod_name)

In [9]:
# with open("clear_product_desc_dict.pickle", 'wb') as f:
#     pickle.dump(prod_desc, f)
# with open("clear_product_intro_dict.pickle", 'wb') as f:
#     pickle.dump(prod_intro, f)
# with open("clear_product_name_dict.pickle", 'wb') as f:
#     pickle.dump(prod_name, f)

In [10]:
# with open("seg_desc.pickle", 'rb') as f:
#     seg_desc = pickle.load(f)
# with open("seg_intro.pickle", 'rb') as f:
#     seg_intro = pickle.load(f)
# with open("seg_name.pickle", 'rb') as f:
#     seg_name = pickle.load(f)

In [11]:
# seg_desc

In [12]:
doc2Vec_model = gensim.models.Doc2Vec.load("./doc2vec_model/doc2vec.model")

In [13]:
def load_json(file):
    with open(file) as jsonfile:
        data = json.load(jsonfile)
    return data

In [14]:
class Embeddings(torch.nn.Module): 
    def __init__(self, w2vmodel): 
        super().__init__() 
        self.weights = torch.FloatTensor(w2vmodel.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(self.weights, padding_idx = -1)
        self.embedding.requires_grad = False
        
        # vector for oov 
        self.oov = torch.nn.Parameter(data=torch.rand(1,250)) 
        self.oov_index = -1 
        self.dim = 250 

    def create_embedding(self, arr): 
        N = arr.shape[0] 
        mask =  (arr==self.oov_index).long() 
        mask_ = mask.unsqueeze(dim=1).float() 
        embed =(1-mask_)*self.embedding((1-mask)*arr) + mask_*(self.oov.expand((N,self.dim))) 
        return embed

In [15]:
w2vmodel = gensim.models.Word2Vec.load('./word2vec/word2vec.model')
embedding_model = Embeddings(w2vmodel)

In [16]:
def LoadDocSentEmb(doc_list, doc_idx, sent_num):
    """ Return the embedding of each sentence of a document. If sentence number is less than sent_num, the last part will
        be filled by 0.
    parameter:
        doc_list: every product intro
        doc_idx : load which product intro
    return: sentence embedding array
    """
    doc_sent = doc_list[doc_idx]
    sent_emb_array = []
    
    for i in range(sent_num):
        id_seq = []
        if i < len(doc_sent):
            sent = doc_sent[i].split()
            for word in sent:
                try:
                    word_id = w2vmodel.wv.vocab[word].index
                except KeyError:
                    word_id = -1  # deal with OOV
                finally:
                    id_seq.append(word_id)
            sent_emb = embedding_model.create_embedding(torch.tensor(id_seq))
        else:
            sent_emb = torch.FloatTensor([np.zeros((250,))])
        sent_emb_array.append(sent_emb.view(len(sent_emb), 1, 250))
    
    return sent_emb_array

In [46]:
class GraphLoader(data.Dataset):
    def __init__(self, mode, root, sent_num=20, sent_len=30, hidden_size=250):
        self.root = root
        self.inputdata = load_json(root+'com_product_intro_word_articles_'+mode+'.json')
        self.golddata = load_json(root+'com_product_name_word_flatten_intro_'+mode+'.json')
        self.golddata = self.trans_data(self.golddata)
        self.sent_len = sent_len
        self.sent_num = sent_num
        self.hidden_size = hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.lstm = nn.LSTM(input_size=250, hidden_size=250, batch_first=True)
        
    def __len__(self):
        """return the size of dataset"""
        return len(self.inputdata)
    
    def __getitem__(self, index):
        # training
        graph = self.create_graph(index) 
        embedding = self.create_lstm_embedding(self.inputdata, index)
        np_embedding = []
        for i in embedding:
            np_embedding.append(torch.flatten(i).detach().numpy())
        
        # ground truth
        gold = self.compute_rouge(self.inputdata,self.golddata, index)
        gold = torch.FloatTensor(gold)
        
        return graph, np_embedding, gold
    
    def create_graph(self, idx):
        """Using doc2vec cosine similarity"""
        sentences_list = self.inputdata[idx]
        sentence_vectors = []
        for i in range(self.sent_num):
            if i < len(sentences_list):
                v = doc2Vec_model.infer_vector(sentences_list[i].split())
            else:
                v = np.zeros((250,)) 
            sentence_vectors.append(v)
        sentence_similarity_martix = np.eye(self.sent_num)
        for j in range(self.sent_num):
            for k in range(self.sent_num):
                if j != k:
                    sentence_similarity_martix[j][k] = cosine_similarity(sentence_vectors[j].reshape(1,250), sentence_vectors[k].reshape(1,250))[0,0]

#         sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
        return sentence_similarity_martix
#     def create_graph(self, idx):
#         """Using word2vec cosine smilarity"""
#         sentences_list = self.inputdata[idx]
#         sentence_vectors = []
#         for i in range(self.sent_num):
#             if i < len(sentences_list):
#                 s = sentences_list[i]
#                 if len(s) != 0:
#                     v = sum([word_embeddings.get(w, np.zeros((250,))) for w in s])/(len(s)+0.001)
#                 else:
#                     v = np.zeros((250,))
#             else:
#                 v = np.zeros((250,)) 
#             sentence_vectors.append(v)
#         sentence_similarity_martix = np.eye(self.sent_num)
#         for j in range(self.sent_num):
#             for k in range(self.sent_num):
#                 if j != k:
#                     sentence_similarity_martix[j][k] = cosine_similarity(sentence_vectors[j].reshape(1,250), sentence_vectors[k].reshape(1,250))[0,0]

# #         sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
#         return sentence_similarity_martix
    
    def create_lstm_embedding(self, data, idx):
        doc_emb_array = LoadDocSentEmb(data, idx, self.sent_num)
        hidden = (torch.randn(1, 1, 250), torch.randn(1, 1, 250))
        doc_hidden = []
        for i in doc_emb_array:
            for j in i:
                out, hidden = self.lstm(j.view(1, 1, -1), hidden)
            doc_hidden.append(hidden[0])
        return doc_hidden
            
    def create_gru_embedding(self, data, idx):
        doc_emb_array = LoadDocSentEmb(data, idx, self.sent_num)
        hidden = torch.randn(1, 1, 250)
        doc_hidden = []
        for i in doc_emb_array:
            for j in i:
                out, hidden = self.gru(j.view(1, 1, -1), hidden)
            doc_hidden.append(hidden[0])
        return doc_hidden
    
    def compute_rouge(self, input, gold, idx):
        score_list = []
        for sentence in input[idx]:
            score = rouge.get_scores(gold[idx], [sentence])
            score_list.append([score[0]['rouge-1']['r']])
            if len(score_list) == self.sent_num:
                break
        if len(score_list) < self.sent_num:
            while len(score_list) != self.sent_num:
                score_list.append([0.0])
        return score_list
    
    def trans_data(self, data):
        trans = []
        for sentences in data:
            s_str = ""
            for w in sentences:
                s_str += w +' '
            trans.append([s_str])
        return trans

In [47]:
A = GraphLoader('train','./').__getitem__(0)

In [48]:
A[0]

array([[ 1.        ,  0.28149128,  0.20456405,  0.40891266,  0.35868987,
         0.21233541,  0.54337847, -0.01039637,  0.31497425,  0.11994876,
         0.26313934,  0.25882566,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.28149128,  1.        ,  0.30849081,  0.34663281,  0.29264575,
         0.26101157,  0.48743409, -0.0782038 ,  0.37751514,  0.07673453,
         0.28535864,  0.20904903,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.20456405,  0.30849081,  1.        ,  0.40339267,  0.18138014,
         0.29245347,  0.3278963 ,  0.10261761,  0.36584604,  0.01906895,
         0.13180581,  0.15425888,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.40891266,  0.34663281,  0.40339267,  1.        ,  0.48009443,
         0.28426003,  0.55666906, -0.02395857,  

In [49]:
A[1]

[array([ 6.31933957e-02,  1.49516881e-01, -1.29103392e-01,  4.47220206e-01,
        -3.34895775e-02,  2.39721686e-02,  1.29734054e-01, -1.50395423e-01,
        -2.43576527e-01,  2.49085113e-01,  4.57347594e-02,  3.76762748e-02,
         1.88841075e-02,  2.76945475e-02,  2.41609588e-01,  2.31583908e-01,
         1.67641297e-01, -2.47901782e-01, -1.05697140e-01,  2.86447376e-01,
         2.97200114e-01,  1.35262638e-01, -1.19395658e-01, -7.18612447e-02,
         2.95265287e-01,  1.56283006e-01, -3.71563472e-02,  5.58733344e-02,
        -7.43893757e-02, -1.53325394e-01,  1.04666181e-01, -2.63756156e-01,
        -1.79716066e-01,  4.73261848e-02,  5.73476180e-02, -1.36448070e-01,
         2.43621975e-01, -2.94630080e-01, -2.49278545e-01, -1.74665302e-01,
         7.43483454e-02,  3.07413265e-02,  1.71803668e-01,  3.49915355e-01,
        -1.87622961e-02,  1.75151289e-01,  4.91692238e-02,  1.90631837e-01,
        -2.18613856e-02, -2.62579590e-01, -2.41618156e-01, -5.55053502e-02,
        -6.5

In [28]:
A[2]

tensor([[0.0476],
        [0.1000],
        [0.1538],
        [0.1852],
        [0.1860],
        [0.2632],
        [0.4667],
        [0.0588],
        [0.3750],
        [0.4000],
        [0.4054],
        [0.2093],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000]])